In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
%set_env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [12]:
import sys
import collections
import cv2
import os
import math
import random
import pickle
import copy
import numpy as np
import pickle
sys.path.append("/home/tanvikamble/MeronymNet-PyTorch/src")

In [13]:
import torch
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse
from torch_geometric.data import DataLoader
import torch.utils.data as data_utils
from torch.utils.tensorboard import SummaryWriter

In [14]:
from losses import BoxVAE_losses as loss
from components.AutoEncoder import GCNAutoEncoder_Combined_Parts
from components.Decoder import Decoder

In [15]:
import matplotlib.pyplot as plt

In [16]:
# from mask_generation import masked_sketch

In [17]:
colors = [(1, 0, 0),
          (0.737, 0.561, 0.561),
          (0.255, 0.412, 0.882),
          (0.545, 0.271, 0.0745),
          (0.98, 0.502, 0.447),
          (0.98, 0.643, 0.376),
          (0.18, 0.545, 0.341),
          (0.502, 0, 0.502),
          (0.627, 0.322, 0.176),
          (0.753, 0.753, 0.753),
          (0.529, 0.808, 0.922),
          (0.416, 0.353, 0.804),
          (0.439, 0.502, 0.565),
          (0.784, 0.302, 0.565),
          (0.867, 0.627, 0.867),
          (0, 1, 0.498),
          (0.275, 0.51, 0.706),
          (0.824, 0.706, 0.549),
          (0, 0.502, 0.502),
          (0.847, 0.749, 0.847),
          (1, 0.388, 0.278),
          (0.251, 0.878, 0.816),
          (0.933, 0.51, 0.933),
          (0.961, 0.871, 0.702)]
colors = (np.asarray(colors)*255)
canvas_size = 660

def plot_bbx(bbx):
    bbx = bbx*canvas_size
    canvas = np.ones((canvas_size,canvas_size,3), np.uint8) * 255
    for i, coord in enumerate(bbx):
        x_minp, y_minp,x_maxp , y_maxp= coord
        if [x_minp, y_minp,x_maxp , y_maxp]!=[0,0,0,0]:
            cv2.rectangle(canvas, (int(x_minp), int(y_minp)), (int(x_maxp) , int(y_maxp) ), colors[i], 6)
    return canvas

# def plot_bbx(bbx):
#     bbx = bbx*canvas_size
#     canvas = np.ones((canvas_size,canvas_size,3), np.uint8) * 255
#     for i, coord in enumerate(bbx):
#         x, y, w ,h = coord
#         if [x, y, w ,h]!=[0,0,0,0]:
#             cv2.rectangle(canvas, (int(x), int(y)), (int(x + w) , int(y + h) ), colors[i], 6)
#     return canvas


In [18]:
def inference(decoder, nodes, obj_class, latent_dims, batch_size):

    decoder.cuda()
    z_latent = torch.normal(torch.zeros([batch_size,latent_dims])).cuda()
    nodes = torch.reshape(nodes,(batch_size,decoder.num_nodes))
    obj_class = torch.reshape(obj_class,(batch_size, decoder.class_size))
    conditioned_z = torch.cat([nodes, z_latent],dim=-1)
    conditioned_z = torch.cat([obj_class, conditioned_z],dim=-1)
    
    x_bbx, x_lbl = decoder(conditioned_z)
    if x_lbl == None:
        return x_bbx,z_latent
    else:
        return x_bbx, x_lbl, z_latent

In [19]:
outfile = '/home/tanvikamble/MeronymNet-PyTorch/src/processed_data/combined_parts/X_train.np'
with open(outfile, 'rb') as pickle_file:
    X_train = pickle.load(pickle_file)

outfile = '/home/tanvikamble/MeronymNet-PyTorch/src/processed_data/combined_parts/class_v.np'
with open(outfile, 'rb') as pickle_file:
    class_v = pickle.load(pickle_file)

outfile = '/home/tanvikamble/MeronymNet-PyTorch/src/processed_data/combined_parts/adj_train.np'
with open(outfile, 'rb') as pickle_file:
    adj_train = pickle.load(pickle_file)

outfile = '/home/tanvikamble/MeronymNet-PyTorch/src/processed_data/combined_parts/X_train_val.np'
with open(outfile, 'rb') as pickle_file:
    X_train_val = pickle.load(pickle_file)

outfile = '/home/tanvikamble/MeronymNet-PyTorch/src/processed_data/combined_parts/class_v_val.np'
with open(outfile, 'rb') as pickle_file:
    class_v_val = pickle.load(pickle_file)
    
outfile = '/home/tanvikamble/MeronymNet-PyTorch/src/processed_data/combined_parts/adj_train_val.np'
with open(outfile, 'rb') as pickle_file:
    adj_train_val = pickle.load(pickle_file)
    
# outfile = '/home/tanvikamble/MeronymNet-PyTorch/src/processed_data/only_torso_experiment_2/X_test.np'
# with open(outfile, 'rb') as pickle_file:
#     X_test = pickle.load(pickle_file)

# outfile = '/home/tanvikamble/MeronymNet-PyTorch/src/processed_data/only_torso_experiment/adj_test.np'
# with open(outfile, 'rb') as pickle_file:
#     adj_test = pickle.load(pickle_file)
    
# outfile = '/home/tanvikamble/MeronymNet-PyTorch/src/processed_data/only_torso_experiment/class_v_test.np'
# with open(outfile, 'rb') as pickle_file:
#     class_v_test = pickle.load(pickle_file)

In [20]:
print(X_train.shape)
print(class_v.shape)
print(adj_train.shape)
# print(X_test.shape)

(50910, 16, 5)
(50910, 7)
(50910, 16, 16)


In [26]:
X_train[X_train<=0] = 0
X_train_val[X_train_val<=0] = 0
# X_test[X_test<=0] = 0

X_train[X_train>=1] = 1
X_train_val[X_train_val>=1] = 1
# X_test[X_test>=1] = 1

# For scaling data to small canvases

In [56]:
def get_limits(X_train):
    X_train_new_limits = np.zeros((X_train.shape[0],4))
    for i in range(X_train.shape[0]):
        min_x=660
        max_x=0
        min_y=660
        max_y=0
        for j in range(16):
            if X_train[i][j][0] == 1:
                min_x = min(min_x,min(X_train[i][j][1]*660,X_train[i][j][3]*660))
                min_y = min(min_y,min(X_train[i][j][2]*660,X_train[i][j][4]*660))

                max_x = max(max_x,max(X_train[i][j][1]*660,X_train[i][j][3]*660))
                max_y = max(max_y,max(X_train[i][j][2]*660,X_train[i][j][4]*660))
        X_train_new_limits[i][0] = min_x
        X_train_new_limits[i][1] = min_y
        X_train_new_limits[i][2] = max_x
        X_train_new_limits[i][3] = max_y
    return X_train_new_limits

In [57]:
def get_new_coordinates(X_train,X_train_new_limits,final_canvas_size_x,final_canvas_size_y):
    X_train_new_coordinates = np.zeros((X_train.shape[0],16,5))
    for i in range(X_train.shape[0]):
        min_x=X_train_new_limits[i][0]
        min_y=X_train_new_limits[i][1]
        max_x=X_train_new_limits[i][2]
        max_y=X_train_new_limits[i][3]

    #     print(max_x - min_x , max_y - min_y)
        for j in range(16):
            if X_train[i][j][0] == 1:
                X_train_new_coordinates[i][j][0] = X_train[i][j][0]
                # Scaling X
                X_train_new_coordinates[i][j][1] = int(final_canvas_size_x * float(((X_train[i][j][1]*660) - min_x) / (max_x - min_x)))
                X_train_new_coordinates[i][j][3] = int(final_canvas_size_x * float(((X_train[i][j][3]*660) - min_x) / (max_x - min_x)))
                #Scaling Y
                X_train_new_coordinates[i][j][2] = int(final_canvas_size_y * float(((X_train[i][j][2]*660) - min_y) / (max_y - min_y)))
                X_train_new_coordinates[i][j][4] = int(final_canvas_size_y * float(((X_train[i][j][4]*660) - min_y) / (max_y - min_y)))
    return X_train_new_coordinates

In [58]:
final_canvas_size_x = 100
final_canvas_size_y = 100

In [59]:
X_train_new_limits = get_limits(X_train)
print(X_train_new_limits)
X_train_new_coordinates = get_new_coordinates(X_train,X_train_new_limits,final_canvas_size_x,final_canvas_size_y)



X_train_val_new_limits = get_limits(X_train_val)
X_train_val_new_coordinates = get_new_coordinates(X_train_val,X_train_val_new_limits,final_canvas_size_x,final_canvas_size_y)

[[312.    305.    347.    354.   ]
 [224.    283.    436.    377.   ]
 [185.    222.    474.    438.   ]
 ...
 [184.55  158.3   475.4   501.65 ]
 [195.725 171.35  465.8   490.175]
 [187.275 161.65  471.2   496.825]]


In [39]:
# for i in range(5):
#     for j in range(16):
#         if X_train[i][j][0] == 1:
#             print(X_train[i][j]*660)
#             print("xxxxxxxxxxxxxxxxxxxx")
#     print(X_train_new_limits[i])
#     print("xxxxxxxxxxxxxxxxxxxx")
#     print("End of This Object")

In [60]:
X_train_new_limits = X_train_new_limits/660
for i in range(X_train_new_coordinates.shape[0]):
    for j in range(16):
        for k in range(1,4):
            X_train_new_coordinates[i][j][k] = X_train_new_coordinates[i][j][k]/100 

In [67]:
X_train_val_new_limits = X_train_val_new_limits/660
for i in range(X_train_val_new_coordinates.shape[0]):
    for j in range(16):
        for k in range(1,4):
            X_train_val_new_coordinates[i][j][k] = X_train_val_new_coordinates[i][j][k]/100 

In [68]:
print(X_train_val_new_limits[100])
print(X_train_val_new_coordinates[10])
print(X_train_val_new_coordinates.dtype)

[0.39090909 0.42575758 0.60909091 0.57272727]
[[1.0e+00 0.0e+00 3.1e-01 3.2e-01 9.7e+01]
 [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00]
 [1.0e+00 8.0e-02 0.0e+00 9.7e-01 8.1e+01]
 [1.0e+00 1.0e-01 3.1e-01 2.3e-01 4.8e+01]
 [1.0e+00 2.0e-01 7.2e-01 4.4e-01 9.1e+01]
 [1.0e+00 2.1e-01 8.0e-01 4.6e-01 9.4e+01]
 [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00]
 [1.0e+00 7.4e-01 4.3e-01 9.0e-01 7.6e+01]
 [1.0e+00 6.3e-01 6.9e-01 8.5e-01 1.0e+02]
 [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00]
 [1.0e+00 6.0e-01 5.6e-01 7.9e-01 8.9e+01]
 [1.0e+00 8.9e-01 7.0e-02 1.0e+00 6.9e+01]
 [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00]]
float64


In [34]:
# for i in range(0,X_train.shape[0],100):
#     print("Start")
#     print("Min and Max Coordinates")
#     print(X_train_new_limits[i][0],X_train_new_limits[i][1],X_train_new_limits[i][2],X_train_new_limits[i][3])
#     for j in range(16):
#         if X_train[i][j][0] == 1:
#             print("Before Scaling Coordinates: ")
#             print(X_train[i][j][1]*660,X_train[i][j][2]*660,X_train[i][j][3]*660,X_train[i][j][4]*660)
#         if X_train_new_coordinates[i][j][0] == 1:
#             print("After Scaling coordinates: ")
#             print(X_train_new_coordinates[i][j][1], X_train_new_coordinates[i][j][2] , X_train_new_coordinates[i][j][3], X_train_new_coordinates[i][j][4])
#     print("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")
#     break
            

# End of scaling to small canvas experiment
- X_train_new_limits are the new limits we need
- X_train_new_coordintates are the new coordinates